In [1]:
import sqlite3

letter_value = {
    "A": 10, "B": 11, "C": 12, "D": 13, "E": 14, "F": 15, "G": 16, "H": 17,
    "I": 34, "J": 18, "K": 19, "L": 20, "M": 21, "N": 22, "O": 35, "P": 23,
    "Q": 24, "R": 25, "S": 26, "T": 27, "U": 28, "V": 29, "W": 32, "X": 30,
    "Y": 31, "Z": 33
}

conn = sqlite3.connect("ID_data.db")
cursor = conn.cursor()
cursor.execute("SELECT ID FROM ID_table")
rows = cursor.fetchall()

gender_map = {'1': '男性', '2': '女性','8': '男性', '9': '女性'}
county_map = {
    "A": "臺北市", "B": "臺中市", "C": "基隆市", "D": "臺南市",
    "E": "高雄市", "F": "新北市", "G": "宜蘭縣", "H": "桃園市",
    "I": "嘉義市", "J": "新竹縣", "K": "苗栗縣", "L": "臺中縣",
    "M": "南投縣", "N": "彰化縣", "O": "新竹市", "P": "雲林縣",
    "Q": "嘉義縣", "R": "臺南縣", "S": "高雄縣", "T": "屏東縣",
    "U": "花蓮縣", "V": "臺東縣", "W": "金門縣", "X": "澎湖縣",
    "Y": "陽明山管理局", "Z": "連江縣"
}
for row in rows:
    id_value = row[0]
    first_char = id_value[0].upper()
    second_digit = id_value[1]
    third_digit = id_value[2]
    if first_char not in county_map or second_digit not in gender_map or third_digit not in '0123456789':
        cursor.execute("DELETE FROM ID_table WHERE ID = ?", (id_value,))
    else:
        cursor.execute(
            "UPDATE ID_table SET country = ? WHERE ID = ?",
            (county_map[first_char], id_value)
        )
        cursor.execute(
            "UPDATE ID_table SET gender = ? WHERE ID = ?",
            (gender_map[second_digit], id_value)
        )

def calculate_check_digit(id_9):
    if len(id_9) != 9:
        return None
    first_char = id_9[0]
    if first_char not in letter_value:
        return None
    if not id_9[1:].isdigit():
        return None

    code = letter_value[first_char]
    a1 = code // 10
    a2 = code % 10
    digits = [int(d) for d in id_9[1:]]

    for last in range(10):
        N = (
            a1 * 1 + a2 * 9 +
            digits[0] * 8 + digits[1] * 7 + digits[2] * 6 +
            digits[3] * 5 + digits[4] * 4 + digits[5] * 3 +
            digits[6] * 2 + digits[7] * 1 + last * 1
        )
        if N % 10 == 0:
            return str(last)
    return None


for (id_value,) in rows:
    id_value = id_value.upper()

    if len(id_value) == 9:
        check_digit = calculate_check_digit(id_value)
        if check_digit:
            new_id = id_value + check_digit
            cursor.execute("UPDATE ID_table SET ID = ? WHERE ID = ?", (new_id, id_value))
            id_value = new_id
        else:
            cursor.execute("DELETE FROM ID_table WHERE ID = ?", (id_value,))
            continue
    elif len(id_value) != 10:
        cursor.execute("DELETE FROM ID_table WHERE ID = ?", (id_value,))
        continue

    code = letter_value[id_value[0]]
    a1 = code // 10
    a2 = code % 10
    digits = [int(d) for d in id_value[1:]]
    N = (
        a1 * 1 + a2 * 9 +
        digits[0] * 8 + digits[1] * 7 + digits[2] * 6 +
        digits[3] * 5 + digits[4] * 4 + digits[5] * 3 +
        digits[6] * 2 + digits[7] * 1 + digits[8] * 1
    )
    if N % 10 != 0:
        cursor.execute("DELETE FROM ID_table WHERE ID = ?", (id_value,))
        continue
    third_digit = id_value[2]
    if third_digit in '012345':
        citizenship = 'taiwan'
    elif third_digit == '6':
        citizenship = 'foreigner'
    elif third_digit == '7':
        citizenship = 'no citizenship'
    elif third_digit == '8':
        citizenship = 'hk mac'
    elif third_digit == '9':
        citizenship = 'china'
    if citizenship:
        cursor.execute(
            "UPDATE ID_table SET citizenship = ? WHERE ID = ?",
            (citizenship, id_value)
        )
conn.commit()
conn.close()
